In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-8475d0kc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8475d0kc
  Resolved https://github.com/huggingface/transformers.git to commit d337631b915ceb4ec18f3a9a1e807603fb7c9b71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=7007032 sha256=37ea42853b81132ad7d676bf5ee128f0aea0ca088762e3a9a27bde7f42a78979
  Stored in directory: /tmp/pip-ephem-wheel-cache-g0_xdte3/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512

In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
import json
import gc

In [ ]:
df = pd.read_csv("/content/train.tsv", sep='\t')
df = df.drop(['topic_id','topic_desc', 'facet_id','facet_desc','question_id','question','answer'], axis=1)
df['clarification_need'] = df['clarification_need'].map({1.0:'Clarification need 1', 2.0:'Clarification need 2', 3.0:'Clarification need 3', 4.0:'Clarification need 4'})
df = df.rename(columns={'initial_request': 'text', 'clarification_need': 'category'})
df.head()

,text,category
0,Tell me about Obama family tree.,Clarification need 2
1,Tell me about Obama family tree.,Clarification need 2
2,Tell me about Obama family tree.,Clarification need 2
3,Tell me about Obama family tree.,Clarification need 2
4,Tell me about Obama family tree.,Clarification need 2


In [ ]:
df['category'].unique()

array(['Clarification need 2', 'Clarification need 1',
       'Clarification need 3', 'Clarification need 4'], dtype=object)

In [ ]:
df['encoded_cat'] = df['category'].astype('category').cat.codes
df.head()

,text,category,encoded_cat
0,Tell me about Obama family tree.,Clarification need 2,1
1,Tell me about Obama family tree.,Clarification need 2,1
2,Tell me about Obama family tree.,Clarification need 2,1
3,Tell me about Obama family tree.,Clarification need 2,1
4,Tell me about Obama family tree.,Clarification need 2,1


In [ ]:
data_texts = df["text"].to_list() # Features (not-tokenized yet)
data_labels = df["encoded_cat"].to_list() # Lables

In [ ]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=0)

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_99', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16,
          validation_data=val_dataset.shuffle(1000).batch(16))

Epoch 1/3
455/455 [==============================] - 58s 73ms/step - loss: 0.1535 - accuracy: 0.9452 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 2/3
455/455 [==============================] - 26s 56ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 3.9135e-04 - val_accuracy: 1.0000
Epoch 3/3
455/455 [==============================] - 25s 55ms/step - loss: 3.9219e-04 - accuracy: 1.0000 - val_loss: 1.7346e-04 - val_accuracy: 1.0000


In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    eval_steps = 10
)

with training_args.strategy.scope():
    trainer_model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

trainer = TFTrainer(
    model=trainer_model,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_239']
You should probably TRAIN this model on a down-stream task to be able to use 

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

{'eval_loss': 0.0007524868023806605}

In [ ]:
save_directory = "/saved_models" # change this to your preferred location

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/saved_models/tokenizer_config.json',
 '/saved_models/special_tokens_map.json',
 '/saved_models/vocab.txt',
 '/saved_models/added_tokens.json')

In [ ]:
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /saved_models were not used when initializing TFDistilBertForSequenceClassification: ['dropout_99']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /saved_models and are newly initialized: ['dropout_259']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_text = test_texts[7]
test_text

'Tell me more on health clubs in Arkansas'

In [ ]:
predict_input = loaded_tokenizer.encode(test_text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

output = loaded_model(predict_input)[0]

prediction_value = tf.argmax(output, axis=1).numpy()[0]
prediction_value

1